In [1]:
import pandas as pd
import numpy as np
df1 = pd.read_csv('data_motores.csv')
pd.set_option('display.max_columns', None)

In [2]:
df1["Eficiencia (%)"]               = df1["Eficiencia (%)"]/100
df1["Factor de carga maximo"]       = df1["Factor de carga maximo"]/100
df1['Potencia Entrada (W)']         = (df1['Potencia (HP)'] * 746 * (df1['Factor de carga maximo'])) / (df1['Eficiencia (%)'])
df1['Corriente a 220V (A)']         = df1['Potencia Entrada (W)'] / (np.sqrt(3) * 220 * df1['Factor de potencia'])
df1['Corriente a 380V (A)']         = df1['Potencia Entrada (W)'] / (np.sqrt(3) * 380 * df1['Factor de potencia'])
df1['Corriente a 220V (A) +25%']    = df1['Corriente a 220V (A)']*1.25
df1['Corriente a 380V (A) +25%']    = df1['Corriente a 380V (A)']*1.25
df1['Velocidad síncrona (RPM)']     = 3600

In [18]:
catalogo                            = pd.read_csv('catalogo_motores.csv')
catalogo['Potencia HP']             = catalogo['Potencia HP'].astype(int)
df2                                 = pd.merge(df1, catalogo[['Potencia HP', 'Velocidad rpm']], left_on='Potencia (HP)', right_on='Potencia HP', how='left')
df2.drop(columns=['Potencia HP'], inplace=True)
df2                                 = df2.rename(columns={'Velocidad rpm': 'Velocidad real (RPM)'})
df2['Deslizamiento (RPM)']          = df2['Velocidad síncrona (RPM)'] - df2['Velocidad real (RPM)'] 
df2['Deslizamiento (%)']            = ((df2['Velocidad síncrona (RPM)'] - df2['Velocidad real (RPM)']) / df2['Velocidad síncrona (RPM)'])*100
df2['Torque (N.m)']                 = (df2['Potencia (HP)'] * 746 * (df1['Factor de carga maximo'])) / ((df2['Velocidad real (RPM)'] * 2 * np.pi)/60)
df2['theta (°)']                    = np.arccos(df2['Factor de potencia'])
df2['Real Corriente 220V']          = df2['Corriente a 220V (A) +25%'] * np.cos(df2['theta (°)']) 
df2['Imaginaria Corriente 220V']    = df2['Corriente a 220V (A) +25%'] * np.sin(df2['theta (°)'])
df2['Real Corriente 380V']          = df2['Corriente a 380V (A) +25%'] * np.cos(df2['theta (°)']) 
df2['Imaginaria Corriente 380V']    = df2['Corriente a 380V (A) +25%'] * np.sin(df2['theta (°)'])
df2["kw/año"]                       = (df2["Potencia Entrada (W)"]/1000) * 6696
df2["costo/año"]                    = df2["kw/año"] * 1009.04

df2["cambio FC"]                    = np.where(df2["Factor de carga maximo"] < 0.8, "Si", "No")
df2["cambio Eficiencia"]            = np.where(df2["Eficiencia (%)"] < 0.85, "Si", "No")
df2["cambio"]                       = np.where((df2["cambio FC"] == "Si") | (df2["cambio Eficiencia"] == "Si"), "Si", "No")

In [4]:
df2.to_csv('data_motores_procesada.csv', index=False)

# PARA 220V

Magnitud de corriente a 220V para determinar el total de la corriente que se usará para dimensionar el grosor del cable sobre la Planta 1

In [5]:
Corriente_Planta_1_220V = np.sqrt((df2[df2['Planta'] == 'Planta 1']['Real Corriente 220V'].sum())**2 + (df2[df2['Planta'] == 'Planta 1']['Imaginaria Corriente 220V'].sum())**2)
print(f'Magnitud de corriente a 220V para los motores de la planta 1: {Corriente_Planta_1_220V} A')

Magnitud de corriente a 220V para los motores de la planta 1: 741.2362705890761 A


Magnitud de corriente a 220V para determinar el total de la corriente que se usará para dimensionar el grosor del cable sobre la Planta 2

In [6]:
Corriente_Planta_2_220V = np.sqrt((df2[df2['Planta'] == 'Planta 2']['Real Corriente 220V'].sum())**2 + (df2[df2['Planta'] == 'Planta 2']['Imaginaria Corriente 220V'].sum())**2)
print(f'Magnitud de corriente a 220V para los motores de la planta 2: {Corriente_Planta_2_220V} A')

Magnitud de corriente a 220V para los motores de la planta 2: 1057.1743551531426 A


Magnitud de corriente a 220V para determinar el total de la corriente que se usará para dimensionar el grosor del cable sobre la Planta 3

In [7]:
Corriente_Planta_3_220V = np.sqrt((df2[df2['Planta'] == 'Planta 3']['Real Corriente 220V'].sum())**2 + (df2[df2['Planta'] == 'Planta 3']['Imaginaria Corriente 220V'].sum())**2)
print(f'Magnitud de corriente a 220V para los motores de la planta 3: {Corriente_Planta_3_220V} A')

Magnitud de corriente a 220V para los motores de la planta 3: 1184.1896564845972 A


Magnitud de corriente a 220V para determinar el total de la corriente que se usará para dimensionar el grosor del cable sobre la Planta 4

In [8]:
Corriente_Planta_4_220V = np.sqrt((df2[df2['Planta'] == 'Planta 4']['Real Corriente 220V'].sum())**2 + (df2[df2['Planta'] == 'Planta 4']['Imaginaria Corriente 220V'].sum())**2)
print(f'Magnitud de corriente a 220V para los motores de la planta 4: {Corriente_Planta_4_220V} A')

Magnitud de corriente a 220V para los motores de la planta 4: 774.7569545132952 A


Magnitud de corriente a 220V para determinar el total de la corriente que se usará para dimensionar el grosor del cable sobre la Planta 5

In [9]:
Corriente_Planta_5_220V = np.sqrt((df2[df2['Planta'] == 'Planta 5']['Real Corriente 220V'].sum())**2 + (df2[df2['Planta'] == 'Planta 5']['Imaginaria Corriente 220V'].sum())**2)
print(f'Magnitud de corriente a 220V para los motores de la planta 5: {Corriente_Planta_5_220V} A')

Magnitud de corriente a 220V para los motores de la planta 5: 1397.3174229028093 A


# PARA 380V

Magnitud de corriente a 380V para determinar el total de la corriente que se usará para dimensionar el grosor del cable sobre la Planta 1

In [10]:
Corriente_Planta_1_380V = np.sqrt((df2[df2['Planta'] == 'Planta 1']['Real Corriente 380V'].sum())**2 + (df2[df2['Planta'] == 'Planta 1']['Imaginaria Corriente 380V'].sum())**2)
print(f'Magnitud de corriente a 380V para los motores de la planta 1: {Corriente_Planta_1_380V} A')

Magnitud de corriente a 380V para los motores de la planta 1: 429.1367882357809 A


Magnitud de corriente a 380V para determinar el total de la corriente que se usará para dimensionar el grosor del cable sobre la Planta 2

In [11]:
Corriente_Planta_2_380V = np.sqrt((df2[df2['Planta'] == 'Planta 2']['Real Corriente 380V'].sum())**2 + (df2[df2['Planta'] == 'Planta 2']['Imaginaria Corriente 380V'].sum())**2)
print(f'Magnitud de corriente a 380V para los motores de la planta 2: {Corriente_Planta_2_380V} A')

Magnitud de corriente a 380V para los motores de la planta 2: 612.0483108781352 A


Magnitud de corriente a 380V para determinar el total de la corriente que se usará para dimensionar el grosor del cable sobre la Planta 3

In [12]:
Corriente_Planta_3_380V = np.sqrt((df2[df2['Planta'] == 'Planta 3']['Real Corriente 380V'].sum())**2 + (df2[df2['Planta'] == 'Planta 3']['Imaginaria Corriente 380V'].sum())**2)
print(f'Magnitud de corriente a 380V para los motores de la planta 3: {Corriente_Planta_3_380V} A')

Magnitud de corriente a 380V para los motores de la planta 3: 685.5834853331879 A


Magnitud de corriente a 380V para determinar el total de la corriente que se usará para dimensionar el grosor del cable sobre la Planta 4

In [13]:
Corriente_Planta_4_380V = np.sqrt((df2[df2['Planta'] == 'Planta 4']['Real Corriente 380V'].sum())**2 + (df2[df2['Planta'] == 'Planta 4']['Imaginaria Corriente 380V'].sum())**2)
print(f'Magnitud de corriente a 380V para los motores de la planta 4: {Corriente_Planta_4_380V} A')

Magnitud de corriente a 380V para los motores de la planta 4: 448.54349998138144 A


Magnitud de corriente a 380V para determinar el total de la corriente que se usará para dimensionar el grosor del cable sobre la Planta 5

In [14]:
Corriente_Planta_5_380V = np.sqrt((df2[df2['Planta'] == 'Planta 5']['Real Corriente 380V'].sum())**2 + (df2[df2['Planta'] == 'Planta 5']['Imaginaria Corriente 380V'].sum())**2)
print(f'Magnitud de corriente a 380V para los motores de la planta 5: {Corriente_Planta_5_380V} A')

Magnitud de corriente a 380V para los motores de la planta 5: 808.9732448384685 A


## CAMBIO DE MOTORES ##

In [15]:
df_catalogo = pd.read_csv("catalogo_motores_completo.csv")
def cambio(fc,potencia_hp,velocidad_sincronica,potencia_actual):
    #factor de carga, potencia_hp, potencia_actual
    costoXaño = (potencia_actual / 1000) * TOTALHORAS * 1009.04
    df_filtro                           = df_catalogo[(df_catalogo["Potencia HP"] == potencia_hp) & (df_catalogo["rpm"] == velocidad_sincronica)].copy()
    df_filtro['Potencia Entrada (W)']   = (df_filtro ['Potencia HP'] * 746 * (fc)) / (df_filtro ['Eficiencia %']/100)
    df_filtro["kw/año"]                 = (df_filtro["Potencia Entrada (W)"]/1000) * TOTALHORAS
    df_filtro["costo/año"]              = df_filtro["kw/año"] * 1009.04
    df_filtro["costo_anterior"]         = costoXaño
    df_filtro["ahorroXmes"]             = (df_filtro["costo_anterior"] - df_filtro["costo/año"]) / 12
    df_filtro["tiempo_retribucion"]     = df_filtro["costo_Pesos"] / df_filtro["ahorroXmes"]
    return df_filtro

In [17]:
#factor de carga, potencia_hp, potencia_actual
df_cambio = cambio(0.8,50,3600,35105.882352941175)
df_cambio

NameError: name 'TOTALHORAS' is not defined

In [ ]:
df2.head()

,Planta,Distancia,Motor,Potencia (HP),Eficiencia (%),Factor de carga maximo,Factor de potencia,Potencia Entrada (W),Corriente a 220V (A),Corriente a 380V (A),Corriente a 220V (A) +25%,Corriente a 380V (A) +25%,Velocidad síncrona (RPM),Velocidad real (RPM),Deslizamiento (RPM),Deslizamiento (%),Torque (N.m),theta (°),Real Corriente 220V,Imaginaria Corriente 220V,Real Corriente 380V,Imaginaria Corriente 380V,kw/año,costo/año,cambio FC,cambio Eficiencia,cambio
0,Planta 1,200,1,30,0.70,0.8,0.74,25577.142857,90.706206,52.514119,113.382757,65.642649,3600,3540,60,1.666667,48.296781,0.737726,83.903240,76.262021,48.575560,44.151697,171264.548571,1.728128e+08,No,Si,Si
1,Planta 2,300,1,40,0.79,0.8,0.82,30217.721519,96.708479,55.989120,120.885599,69.986399,3600,3565,35,0.972222,63.944126,0.609385,99.126191,69.190507,57.388848,40.057662,202337.863291,2.041670e+08,No,Si,Si
2,Planta 3,350,1,60,0.85,0.6,0.78,31595.294118,106.302748,61.543696,132.878435,76.929620,3600,3570,30,0.833333,71.836389,0.676131,103.645179,83.152602,60.005104,48.140980,211562.089412,2.134746e+08,Si,No,Si
3,Planta 4,300,1,30,0.82,0.6,0.78,16375.609756,55.095936,31.897647,68.869921,39.872059,3600,3540,60,1.666667,36.222586,0.676131,53.718538,43.097385,31.100206,24.951118,109651.082927,1.106423e+08,Si,Si,Si
4,Planta 5,500,1,100,0.91,0.6,0.81,49186.813187,159.360381,92.261273,199.200476,115.326591,3600,3570,30,0.833333,119.727315,0.626644,161.352385,116.817110,93.414539,67.630959,329354.901099,3.323323e+08,Si,No,Si
